### Airlines EDA using Plotly

This is a notebook for doing some basic airlines EDA using plotly

#### Initial Data Prep

In [3]:
airlines = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/201*.parquet")

# Filter to datset with entries where diverted != 1, cancelled != 1, dep_delay != Null, and arr_delay != Null
airlines = airlines.where('DIVERTED != 1') \
                   .where('CANCELLED != 1') \
                   .filter(airlines['DEP_DEL15'].isNotNull())

print("Number of records in full dataset:", airlines.count())

# Generate other Departure Delay outcome indicators for n minutes
def CreateNewDepDelayOutcome(data, thresholds):
  for threshold in thresholds:
    data = data.withColumn('Dep_Del' + str(threshold), (data['Dep_Delay'] >= threshold).cast('integer'))
  return data  
  
airlines = CreateNewDepDelayOutcome(airlines, [30])

def SplitDataset(model_name):
  # Split airlines data into train, dev, test
  test = airlines.where('Year = 2019') # held out
  train, val = airlines.where('Year != 2019').randomSplit([7.0, 1.0], 6)

  # Select a mini subset for the training dataset (~2000 records)
  mini_train = train.sample(fraction=0.0001, seed=6)

  print("train_" + model_name + " size = " + str(train.count()))
  print("mini_train_" + model_name + " size = " + str(mini_train.count()))
  print("val_" + model_name + " size = " + str(val.count()))
  print("test_" + model_name + " size = " + str(test.count()))
  
  return (mini_train, train, val, test) 

mini_train, train, val, test = SplitDataset("")

Number of records in full dataset: 31174076
train_ size = 20919374
mini_train_ size = 2065
val_ size = 2986939
test_ size = 7268232

In [4]:
# Create the full data (not including test) for EDA
full_data = train.union(val)

In [5]:
# save full data as parquet for easier analysis
full_data.write.mode('overwrite').format("parquet").save("dbfs/user/team20/full_training_data_airlines.parquet")
display(dbutils.fs.ls("dbfs/user/team20"))

path,name,size
dbfs:/dbfs/user/team20/DecisionTree-2-dianai-models/,DecisionTree-2-dianai-models/,0
dbfs:/dbfs/user/team20/airilnes_mini_train.avro/,airilnes_mini_train.avro/,0
dbfs:/dbfs/user/team20/airlines-backup1-3-10.parquet/,airlines-backup1-3-10.parquet/,0
dbfs:/dbfs/user/team20/airlines-backup2-3-10.parquet/,airlines-backup2-3-10.parquet/,0
dbfs:/dbfs/user/team20/airlines-backup3-3-10.parquet/,airlines-backup3-3-10.parquet/,0
dbfs:/dbfs/user/team20/airlines_mini_train.avro/,airlines_mini_train.avro/,0
dbfs:/dbfs/user/team20/airlines_mini_train.parquet/,airlines_mini_train.parquet/,0
dbfs:/dbfs/user/team20/airlines_mini_train_dep.parquet/,airlines_mini_train_dep.parquet/,0
dbfs:/dbfs/user/team20/airlines_test.avro/,airlines_test.avro/,0
dbfs:/dbfs/user/team20/airlines_test.parquet/,airlines_test.parquet/,0


#### Start Here!

In [7]:
# Read in data for faster querying
full_data = spark.read.option("header", "true").parquet(f"dbfs/user/team20/full_training_data_airlines.parquet")

In [8]:
outcomeName = 'Dep_Del30'
nfeatureNames = [
  # 0        1            2              3              4                5                6               7               8 
  'Year', 'Month', 'Day_Of_Month', 'Day_Of_Week', 'CRS_Dep_Time', 'CRS_Arr_Time', 'CRS_Elapsed_Time', 'Distance', 'Distance_Group'
]
#                         9              10       11
cfeatureNames = ['Op_Unique_Carrier', 'Origin', 'Dest']

# Filter full data to just relevant columns
full_data_dep = full_data.select([outcomeName] + nfeatureNames + cfeatureNames)

#### Install Dependencies

In [10]:
%sh 
pip install plotly --upgrade

Collecting plotly
 Downloading https://files.pythonhosted.org/packages/15/90/918bccb0ca60dc6d126d921e2c67126d75949f5da777e6b18c51fb12603d/plotly-4.6.0-py2.py3-none-any.whl (7.1MB)
Collecting retrying>=1.3.3 (from plotly)
 Downloading https://files.pythonhosted.org/packages/44/ef/beae4b4ef80902f22e3af073397f079c96969c69b2c7d52a57ea9ae61c9d/retrying-1.3.3.tar.gz
Requirement already satisfied, skipping upgrade: six in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from plotly) (1.12.0)
Building wheels for collected packages: retrying
 Building wheel for retrying (setup.py): started
 Building wheel for retrying (setup.py): finished with status 'done'
 Stored in directory: /root/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
Installing collected packages: retrying, plotly
Successfully installed plotly-4.6.0 retrying-1.3.3

In [11]:
import pandas as pd
import numpy as np
from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import datetime as dt
import plotly.express as px

import plotly.io as pio

In [12]:
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

from pyspark.sql.functions import col, countDistinct

#### Basic Initial EDA

In [14]:
print("Expected Num Records:  23903381")
print("  Actual Num Records: ", full_data_dep.count())

Expected Num Records: 23903381
 Actual Num Records: 23906619

In [15]:
display(full_data_dep.sample(False, 0.00001))

Dep_Del30,Year,Month,Day_Of_Month,Day_Of_Week,CRS_Dep_Time,CRS_Arr_Time,CRS_Elapsed_Time,Distance,Distance_Group,Op_Unique_Carrier,Origin,Dest
0,2016,5,2,1,1922,2241,139.0,833.0,4,OO,LAX,COS
0,2016,5,13,5,730,1003,333.0,2296.0,10,AA,CLT,SFO
0,2016,5,13,5,1825,2110,105.0,616.0,3,WN,BNA,MCO
0,2016,5,15,7,1138,1307,89.0,407.0,2,NK,LAS,OAK
0,2016,5,31,2,610,720,70.0,325.0,2,WN,OAK,BUR
0,2016,10,3,1,1830,2120,170.0,951.0,4,AA,PHL,PBI
0,2016,10,4,2,1145,1351,246.0,1589.0,7,DL,MSP,SFO
0,2016,10,6,4,710,857,107.0,500.0,3,DL,DTW,CLT
0,2016,10,17,1,1220,1440,140.0,693.0,3,EV,ORD,OKC
0,2016,10,21,5,642,1137,175.0,1222.0,5,UA,LAS,IAH


In [16]:
# Get number of distinct values for each column in full training dataset
display(full_data_dep.agg(*(countDistinct(col(c)).alias(c) for c in full_data_dep.columns)))

Dep_Del30,Year,Month,Day_Of_Month,Day_Of_Week,CRS_Dep_Time,CRS_Arr_Time,CRS_Elapsed_Time,Distance,Distance_Group,Op_Unique_Carrier,Origin,Dest
2,4,12,31,7,1423,1440,645,1607,11,19,364,364


#### EDA

##### Helper Functions for EDA

In [19]:
# Helper Function for plotting distinct values of feature on X and number of flights on Y, categorized by outocme variable
def MakeRegBarChart(full_data_dep, outcomeName, var, orderBy, barmode, xtype):
  if (orderBy):
      d = full_data_dep.select(var, outcomeName).groupBy(var, outcomeName).count().orderBy(orderBy).toPandas()
  else:
    d = full_data_dep.select(var, outcomeName).groupBy(var, outcomeName).count().toPandas()

  t1 = go.Bar(
    x = d[d[outcomeName] == 0.0][var],
    y = d[d[outcomeName] == 0.0]["count"],
    name=outcomeName + " = " + str(0.0)
  )
  t2 = go.Bar(
    x = d[d[outcomeName] == 1.0][var],
    y = d[d[outcomeName] == 1.0]["count"],
    name=outcomeName + " = " + str(1.0)
  )

  l = go.Layout(
    barmode=barmode, 
    title="Flight Counts by " + var + " & " + outcomeName,
    xaxis=dict(title=var, type=xtype),
    yaxis=dict(title="Number of Flights")
  )
  fig = go.Figure(data=[t1, t2], layout=l)
  fig.show()

In [20]:
# Helper function for Group 3 graphs that plot the probability of outcome on the x axis, the number of flights on the x axis
# With entries for each distinct value of the feature as separate bars.

def MakeProbBarChart(full_data_dep, outcomeName, var, xtype, numDecimals):
  # Filter out just to rows with delays or no delays
  d_delay = full_data_dep.select(var, outcomeName).filter(col(outcomeName) == 1.0).groupBy(var, outcomeName).count().orderBy("count")
  d_nodelay = full_data_dep.select(var, outcomeName).filter(col(outcomeName) == 0.0).groupBy(var, outcomeName).count().orderBy("count")

  # Join tables to get probabilities of departure delay for each table
  probs = d_delay.join(d_nodelay, d_delay[var] == d_nodelay[var]) \
             .select(d_delay[var], (d_delay["count"]).alias("DelayCount"), (d_nodelay["count"]).alias("NoDelayCount"), \
                     (d_delay["count"] / (d_delay["count"] + d_nodelay["count"])).alias("Prob_" + outcomeName))

  # Join back with original data to get 0/1 labeling with probablities of departure delay as attribute of airlines
  d = full_data_dep.select(var, outcomeName).groupBy(var, outcomeName).count()
  d = d.join(probs, full_data_dep[var] == probs[var]) \
       .select(d[var], d[outcomeName], d["count"], probs["Prob_" + outcomeName]) \
       .orderBy("Prob_" + outcomeName, outcomeName).toPandas()
  d = d.round({'Prob_' + outcomeName: numDecimals})

  t1 = go.Bar(
    x = d[d[outcomeName] == 0.0]["Prob_" + outcomeName],
    y = d[d[outcomeName] == 0.0]["count"],
    name=outcomeName + " = " + str(0.0),
    text=d[d[outcomeName] == 0.0][var]
  )
  t2 = go.Bar(
    x = d[d[outcomeName] == 1.0]["Prob_" + outcomeName],
    y = d[d[outcomeName] == 1.0]["count"],
    name=outcomeName + " = " + str(1.0),
    text=d[d[outcomeName] == 1.0][var]
  )

  l = go.Layout(
    barmode='stack', 
    title="Flight Counts by " + "Prob_" + outcomeName + " & " + outcomeName + " for each " + var,
    xaxis=dict(title="Prob_" + outcomeName + " (Note: axis type = " + xtype + ")", type=xtype),
    yaxis=dict(title="Number of Flights")
  )
  fig = go.Figure(data=[t1, t2], layout=l)
  fig.show()
  
  return d

##### Group 1 Plots

In [22]:
# Plot Year and outcome
var = "Year"
d = full_data_dep.select(var, outcomeName).groupBy(var, outcomeName).count().orderBy(var).toPandas()
display(d)

Year,Dep_Del30,count
2015,0,5062018
2015,1,651501
2016,0,4941371
2016,1,596925
2017,0,4935909
2017,1,643966
2018,1,839613
2018,0,6235316


In [23]:
# Plot Year and outcome
var = "Year"
MakeRegBarChart(full_data_dep, outcomeName, var, orderBy=var, barmode='stack', xtype='category')

In [24]:
# Plot Month & outcome
var = "Month"
d = full_data_dep.select(var, outcomeName).groupBy(var, outcomeName).count().orderBy(var).toPandas()
display(d)

Month,Dep_Del30,count
1,0,1658588
1,1,222466
2,0,1544362
2,1,192578
3,1,219179
3,0,1817701
4,0,1775554
4,1,209059
5,0,1816209
5,1,237335


In [25]:
# Plot Month and outcome
var = "Month"
MakeRegBarChart(full_data_dep, outcomeName, var, orderBy=var, barmode='stack', xtype='category')

In [26]:
# Plot that demonstrates the probability of a departure delay, given the day of year (interaction of month & day of month)
var = "Day_Of_Year"
d = full_data_dep.select("Month", "Day_Of_Month", outcomeName) \
                 .withColumn(var, f.concat(f.col('Month'), f.lit('-'), f.col('Day_Of_Month'))) \
                 .groupBy(var, "Month", "Day_Of_Month", outcomeName).count() \
                 .orderBy("Month", "Day_Of_Month") \
                 .toPandas()
display(d)

Day_Of_Year,Month,Day_Of_Month,Dep_Del30,count
1-1,1,1,1,8181
1-1,1,1,0,49674
1-2,1,2,0,54752
1-2,1,2,1,11578
1-3,1,3,1,11882
1-3,1,3,0,52321
1-4,1,4,1,10975
1-4,1,4,0,50076
1-5,1,5,1,11557
1-5,1,5,0,51344


In [27]:
# Plot Day_Of_Month interacted with Month and outcome
var = "Day_Of_Year"
d = full_data_dep.select("Month", "Day_Of_Month", outcomeName) \
                 .withColumn(var, f.concat(f.col('Month'), f.lit('-'), f.col('Day_Of_Month'))) \
                 .groupBy(var, "Month", "Day_Of_Month", outcomeName).count() \
                 .orderBy("Month", "Day_Of_Month") \
                 .toPandas()

t1 = go.Bar(
  x = d[d[outcomeName] == 0.0][var],
  y = d[d[outcomeName] == 0.0]["count"],
  name=outcomeName + " = " + str(0.0)
)
t2 = go.Bar(
  x = d[d[outcomeName] == 1.0][var],
  y = d[d[outcomeName] == 1.0]["count"],
  name=outcomeName + " = " + str(1.0)
)

l = go.Layout(
  barmode='stack', 
  title="Flight Counts by " + var + " & " + outcomeName,
  xaxis=dict(title=var, type='category'),
  yaxis=dict(title="Number of Flights")
)
fig = go.Figure(data=[t1, t2], layout=l)
fig.show()

In [28]:
# Plot Day of Week and outcome
var = "Day_Of_Week"
d = full_data_dep.select(var, outcomeName).groupBy(var, outcomeName).count().orderBy(var).toPandas()
display(d)

Day_Of_Week,Dep_Del30,count
1,0,3112270
1,1,435414
2,1,374152
2,0,3073706
3,1,369226
3,0,3128827
4,0,3121767
4,1,433808
5,0,3127313
5,1,448833


In [29]:
# Plot Day of Week and outcome
var = "Day_Of_Week"
MakeRegBarChart(full_data_dep, outcomeName, var, orderBy=var, barmode='group', xtype='category')

In [30]:
# Effectively demonstrates the probability of a departure delay, given the distance group
var = "Distance_Group"
d = full_data_dep.select(var, outcomeName).groupBy(var, outcomeName).count().orderBy(var).toPandas()
display(d)

Distance_Group,Dep_Del30,count
1,0,2654988
1,1,311339
2,1,636904
2,0,5021522
3,0,4170295
3,1,529023
4,0,3238593
4,1,430373
5,0,2188947
5,1,319905


In [31]:
# Plot Distance Group and outcome
var = "Distance_Group"
MakeRegBarChart(full_data_dep, outcomeName, var, orderBy=var, barmode='group', xtype='category')

In [32]:
# Effectively demonstrates the probability of a departure delay, given the carrier
# Airline Codes to Airlines: https://www.bts.gov/topics/airlines-and-airports/airline-codes
var = "Op_Unique_Carrier"
d = full_data_dep.select(var, outcomeName).groupBy(var, outcomeName).count().orderBy("count").toPandas()
display(d)

Op_Unique_Carrier,Dep_Del30,count
HA,1,10554
G4,1,13318
US,1,17410
YV,1,26582
9E,1,29688
VX,1,30038
YX,1,33948
OH,1,36232
AS,1,53326
F9,1,66029


In [33]:
# Plot Carrier and outcome
# Airline Codes to Airlines: https://www.bts.gov/topics/airlines-and-airports/airline-codes
var = "Op_Unique_Carrier"
MakeRegBarChart(full_data_dep, outcomeName, var, orderBy='count', barmode='group', xtype='category')

In [34]:
# Plot Origin and outcome
# Airport Codes: https://www.bts.gov/topics/airlines-and-airports/world-airport-codes
var = "Origin"
MakeRegBarChart(full_data_dep, outcomeName, var, orderBy='count', barmode='group', xtype='category')

In [35]:
# Plot Destination and outcome
# Airport Codes: https://www.bts.gov/topics/airlines-and-airports/world-airport-codes
var = "Dest"
MakeRegBarChart(full_data_dep, outcomeName, var, orderBy='count', barmode='group', xtype='category')

In [36]:
# Plot CRS_Elapsed_Time and outcome
var = "CRS_Elapsed_Time"
MakeRegBarChart(full_data_dep, outcomeName, var, orderBy=var, barmode='stack', xtype='linear')

In [37]:
# Plot CRS_Dep_Time and outcome
var = "CRS_Dep_Time"
MakeRegBarChart(full_data_dep, outcomeName, var, orderBy=var, barmode='stack', xtype='linear')

In [38]:
# Plot CRS_Arr_Time and outcome
var = "CRS_Arr_Time"
MakeRegBarChart(full_data_dep, outcomeName, var, orderBy=var, barmode='stack', xtype='linear')

##### Group 2 Plots

In [40]:
# Make helper code for bucketizing (binning) values
from pyspark.ml.feature import Bucketizer
from pyspark.sql.functions import udf
from pyspark.sql.types import *

# Augments the provided dataset for the given variable with binned/bucketized
# versions of that variable, as defined by splits parameter
# Column name suffixed with '_bin' will be the bucketized column
# Column name suffixed with '_binlabel' will be the nicely-named version of the bucketized column
def BinValues(df, var, splits, labels):
  bucketizer = Bucketizer(splits=splits, inputCol=var, outputCol=var + "_bin")
  df_buck = bucketizer.setHandleInvalid("keep").transform(df)
  
  bucketMaps = {}
  bucketNum = 0
  for l in labels:
    bucketMaps[bucketNum] = l
    bucketNum = bucketNum + 1
    
  def newCols(x):
    return bucketMaps[x]
  
  callnewColsUdf = udf(newCols, StringType())
    
  return df_buck.withColumn(var + "_binlabel", callnewColsUdf(f.col(var + "_bin")))


In [41]:
var = 'CRS_Dep_Time'
d = full_data_dep.select(var, outcomeName)
d = BinValues(d, var, splits = [0, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400],
              labels = ['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm',
                        '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'])

MakeRegBarChart(d, outcomeName, var + "_bin", orderBy=var + "_bin", barmode='group', xtype='category')

In [42]:
var = 'CRS_Dep_Time'
d = full_data_dep.select(var, outcomeName, 'Day_Of_Week')
d = BinValues(d, var, splits = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400],
              labels = [str(i) for i in range(0, 24)])

MakeRegBarChart(d, outcomeName, var + "_bin", orderBy=var + "_bin", barmode='stack', xtype='category')

In [43]:
# Plot that demonstrates the probability of a departure delay, given the departure time of week (interaction of binned departure time and day of week)
var = "Dep_Time_Of_Week"
d2 = d.select("Day_Of_Week", "CRS_Dep_Time_bin", outcomeName) \
                 .withColumn(var, f.concat(f.col('Day_Of_Week'), f.lit('-'), f.col('CRS_Dep_Time_bin'))) \
                 .groupBy(var, "Day_Of_Week", "CRS_Dep_Time_bin", outcomeName).count() \
                 .orderBy("Day_Of_Week", "CRS_Dep_Time_bin", ) \
                 .toPandas()
display(d2)

Dep_Time_Of_Week,Day_Of_Week,CRS_Dep_Time_bin,Dep_Del30,count
1-0.0,1,0.0,0,9031
1-0.0,1,0.0,1,1149
1-1.0,1,1.0,0,3104
1-1.0,1,1.0,1,345
1-2.0,1,2.0,0,831
1-2.0,1,2.0,1,165
1-3.0,1,3.0,1,112
1-3.0,1,3.0,0,509
1-4.0,1,4.0,0,325
1-4.0,1,4.0,1,58


##### Group 3 Plots

In [48]:
# Plot Carrier and outcome
# Airline Codes to Airlines: https://www.bts.gov/topics/airlines-and-airports/airline-codes
var = "Op_Unique_Carrier"

# Filter out just to rows with delays or no delays
d_delay = full_data_dep.select(var, outcomeName).filter(col(outcomeName) == 1.0).groupBy(var, outcomeName).count().orderBy("count")
d_nodelay = full_data_dep.select(var, outcomeName).filter(col(outcomeName) == 0.0).groupBy(var, outcomeName).count().orderBy("count")

# Join tables to get probabilities of departure delay for each table
probs = d_delay.join(d_nodelay, d_delay[var] == d_nodelay[var]) \
           .select(d_delay[var], (d_delay["count"]).alias("DelayCount"), (d_nodelay["count"]).alias("NoDelayCount"), \
                   (d_delay["count"] / (d_delay["count"] + d_nodelay["count"])).alias("Prob_" + outcomeName))

# Join back with original data to get 0/1 labeling with probablities of departure delay as attribute of airlines
d = full_data_dep.select(var, outcomeName).groupBy(var, outcomeName).count()
d = d.join(probs, full_data_dep[var] == probs[var]) \
     .select(d[var], d[outcomeName], d["count"], probs["Prob_" + outcomeName]) \
     .orderBy("Prob_" + outcomeName, outcomeName).toPandas()
d = d.round({'Prob_' + outcomeName: 4})

display(d)

Op_Unique_Carrier,Dep_Del30,count,Prob_Dep_Del30
HA,0,305112,0.0334
HA,1,10554,0.0334
AS,0,720162,0.0689
AS,1,53326,0.0689
DL,0,3332487,0.0853
DL,1,310812,0.0853
US,0,176725,0.0897
US,1,17410,0.0897
AA,0,3018227,0.1112
AA,1,377689,0.1112


In [49]:
# Plot Carrier and outcome with bar plots of probability on x axis
# Airline Codes to Airlines: https://www.bts.gov/topics/airlines-and-airports/airline-codes
var = "Op_Unique_Carrier"  
MakeProbBarChart(full_data_dep, outcomeName, var, xtype='category', numDecimals=4)

In [50]:
# Plot Origin airport and outcome with bar plots of probability on x axis
# Airport Codes: https://www.bts.gov/topics/airlines-and-airports/world-airport-codes
var = "Origin"
MakeProbBarChart(full_data_dep, outcomeName, var, xtype='category', numDecimals=4)

In [51]:
# Plot Destination airport and outcome with bar plots of probability on x axis
# Airport Codes: https://www.bts.gov/topics/airlines-and-airports/world-airport-codes
var = "Dest"
MakeProbBarChart(full_data_dep, outcomeName, var, xtype='category', numDecimals=4)

In [52]:
# Plot distance group and outcome with bar plots of probability on x axis
var = "Distance_Group"
MakeProbBarChart(full_data_dep, outcomeName, var, xtype='linear', numDecimals=4)

In [53]:
# Plot Month and outcome with bar plots of probability on x axis
var = "Month"
MakeProbBarChart(full_data_dep, outcomeName, var, xtype='linear', numDecimals=4)

In [54]:
# Plot Day_Of_Year and outcome with bar plots of probability on x axis?
var = "Day_Of_Year"
d = full_data_dep.withColumn(var, f.concat(f.col('Month'), f.lit('-'), f.col('Day_Of_Month')))
d = MakeProbBarChart(d, outcomeName, var, xtype='category', numDecimals=10)

In [55]:
# Plot CRS_Elapsed_Time and outcome with bar plots of probability on x axis
var = "CRS_Elapsed_Time"
MakeProbBarChart(full_data_dep, outcomeName, var, xtype='category', numDecimals=10)

##### Group 4 Plots

In [57]:
def PlotBasicBoxPlots(full_data_dep, var, outcomeName):
  d = full_data_dep.select(var, outcomeName).sample(True, 0.0005, 6).toPandas()
  fig = px.box(d, y=var, points="all", color=outcomeName, title="Boxplots of " + var + " by " + outcomeName)
  fig.show()

In [58]:
var = "Distance"
PlotBasicBoxPlots(full_data_dep, var, outcomeName)

In [59]:
def PlotBinnedBoxPlots(full_data_dep, var, binnedVar, outcomeName):
  d = full_data_dep.select(var, outcomeName, binnedVar).sample(True, 0.0005, 6).toPandas()
  fig = px.box(d, x=binnedVar, y=var, points="all", color=outcomeName)
  fig.show()

In [60]:
var = "Distance"
binnedVar = var + "_Group"
PlotBinnedBoxPlots(full_data_dep, var, binnedVar, outcomeName)

In [61]:
var = "CRS_Elapsed_Time"
PlotBasicBoxPlots(full_data_dep, var, outcomeName)

In [62]:
var = "CRS_Elapsed_Time"
binnedVar = var + "_bin"

d = BinValues(full_data_dep, var, 
              splits = [-100, 60, 120, 180, 240, 300, 360, 420, 480, 540, 600, 660, 720], 
              labels = ['1 hour', '2 hours', '3 hours', '4 hours', '5 hours', '6 hours', '7 hours', '8 hours', '9 hours', '10 hours', '11 hours', '12 hours'])

PlotBinnedBoxPlots(d, var, binnedVar, outcomeName)

In [63]:
# Plot CRS_Elapsed_Time and outcome as porbability chart
var = "CRS_Elapsed_Time"
binnedVar = var + "_bin"

d = BinValues(full_data_dep, var, 
              splits = [-100, 60, 120, 180, 240, 300, 360, 420, 480, 540, 600, 660, 720], 
              labels = ['1 hour', '2 hours', '3 hours', '4 hours', '5 hours', '6 hours', '7 hours', '8 hours', '9 hours', '10 hours', '11 hours', '12 hours'])
MakeRegBarChart(d, outcomeName, binnedVar, orderBy=binnedVar, barmode='stack', xtype='category')

In [64]:
MakeProbBarChart(d, outcomeName, binnedVar + "label", xtype='category', numDecimals=4)

In [65]:
var = "CRS_Dep_Time"
PlotBasicBoxPlots(full_data_dep, var, outcomeName)

In [66]:
var = "CRS_Dep_Time"
binnedVar = var + "_bin"

d = BinValues(full_data_dep, var, 
              splits = [0, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400], 
              labels = ['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'])

PlotBinnedBoxPlots(d, var, binnedVar, outcomeName)

In [67]:
MakeRegBarChart(d, outcomeName, binnedVar, orderBy=binnedVar, barmode='stack', xtype='category')

In [68]:
MakeProbBarChart(d, outcomeName, binnedVar + "label", xtype='category', numDecimals=4)

In [69]:
var = "CRS_Dep_Time"
binnedVar = var + "_bin"

d = BinValues(full_data_dep, var, 
              splits = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 
                        1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400], 
              labels = ['12am-1am', '1am-2am', '2am-3am', '3am-4am', '4am-5am', '5am-6am', 
                        '6am-7am', '7am-8am', '8am-9am', '9am-10am', '10am-11am', '11am-12pm', 
                        '12pm-1pm', '1pm-2pm', '2pm-3pm', '3pm-4pm', '4pm-5pm', '5pm-6pm', 
                        '6pm-7pm', '7pm-8pm', '8pm-9pm', '9pm-10pm', '10pm-11pm', '11pm-12am'])

PlotBinnedBoxPlots(d, var, binnedVar, outcomeName)

In [70]:
MakeRegBarChart(d, outcomeName, binnedVar, orderBy=binnedVar, barmode='stack', xtype='category')

In [71]:
MakeProbBarChart(d, outcomeName, binnedVar + "label", xtype='category', numDecimals=4)

In [72]:
var = "CRS_Arr_Time"
PlotBasicBoxPlots(full_data_dep, var, outcomeName)

In [73]:
var = "CRS_Arr_Time"
binnedVar = var + "_bin"

d = BinValues(full_data_dep, var, 
              splits = [0, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400], 
              labels = ['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'])

PlotBinnedBoxPlots(d, var, binnedVar, outcomeName)

In [74]:
MakeRegBarChart(d, outcomeName, binnedVar, orderBy=binnedVar, barmode='stack', xtype='category')

In [75]:
MakeProbBarChart(d, outcomeName, binnedVar + "label", xtype='category', numDecimals=4)

In [76]:
var = "CRS_Arr_Time"
binnedVar = var + "_bin"

d = BinValues(full_data_dep, var, 
              splits = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 
                        1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400], 
              labels = ['12am-1am', '1am-2am', '2am-3am', '3am-4am', '4am-5am', '5am-6am', 
                        '6am-7am', '7am-8am', '8am-9am', '9am-10am', '10am-11am', '11am-12pm', 
                        '12pm-1pm', '1pm-2pm', '2pm-3pm', '3pm-4pm', '4pm-5pm', '5pm-6pm', 
                        '6pm-7pm', '7pm-8pm', '8pm-9pm', '9pm-10pm', '10pm-11pm', '11pm-12am'])

PlotBinnedBoxPlots(d, var, binnedVar, outcomeName)

In [77]:
MakeRegBarChart(d, outcomeName, binnedVar, orderBy=binnedVar, barmode='stack', xtype='category')

In [78]:
MakeProbBarChart(d, outcomeName, binnedVar + "label", xtype='category', numDecimals=4)

In [80]:
# Plot Day_Of_Month interacted with Month and outcome
var = "Origin_Dest"
d = full_data_dep.select("Origin", "Dest", outcomeName) \
                 .withColumn(var, f.concat(f.col('Origin'), f.lit('-'), f.col('Dest')))

MakeProbBarChart(d, outcomeName, var, xtype='category', numDecimals=10)

### Day_Of_Week interacted with binned CRS_Dep_Time & Dep_Del30

In [82]:
dow_dtime_df = full_data_dep[['DAY_OF_WEEK', 'CRS_DEP_TIME', 'DEP_DEL30']].toPandas()
print(dow_dtime_df.shape)

dow_dtime_df['crs_dep_binned'] = pd.cut(x = dow_dtime_df['CRS_DEP_TIME'], bins = [0, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400], labels = ['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'])

dow_dtime_df['dayOfWeek_binned'] = pd.cut(x = dow_dtime_df['DAY_OF_WEEK'], bins = [0, 1, 2, 3, 4, 5 , 6, 7], labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

dow_dtime_binned_df = dow_dtime_df[['crs_dep_binned', 'dayOfWeek_binned', 'DEP_DEL30']].groupby(['crs_dep_binned', 'dayOfWeek_binned'], as_index = False).count()

(23906619, 3)

In [83]:
import plotly.offline as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_dtime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Monday'].groupby('crs_dep_binned').sum()['DEP_DEL30'].tolist(),
    name='Monday'
    
)
trace2 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_dtime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Tuesday'].groupby('crs_dep_binned').sum()['DEP_DEL30'].tolist(),
    name='Tuesday'
)

trace3 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_dtime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Wednesday'].groupby('crs_dep_binned').sum()['DEP_DEL30'].tolist(),
    name='Wednesday'
    
)
trace4 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_dtime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Thursday'].groupby('crs_dep_binned').sum()['DEP_DEL30'].tolist(),
    name='Thursday'
)

trace5 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_dtime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Friday'].groupby('crs_dep_binned').sum()['DEP_DEL30'].tolist(),
    name='Friday'
    
)

trace6 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_dtime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Saturday'].groupby('crs_dep_binned').sum()['DEP_DEL30'].tolist(),
    name='Saturday'
)

trace7 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_dtime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Sunday'].groupby('crs_dep_binned').sum()['DEP_DEL30'].tolist(),
    name='Sunday'
)

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7]
layout = go.Layout(
    barmode='group', 
    title="Day_Of_Week interacted with binned CRS_Dep_Time & Dep_Del30",
    xaxis=dict(title="Binned CRS Departure Time", type='category'),
    yaxis=dict(title="Number of Delays")
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

#### Day_Of_Week interacted with binned CRS_Arr_Time & Dep_Del30

In [85]:
dow_atime_df = full_data_dep[['DAY_OF_WEEK', 'CRS_ARR_TIME', 'DEP_DEL30']].toPandas()
print(dow_atime_df.shape)

dow_atime_df['crs_arr_binned'] = pd.cut(x = dow_atime_df['CRS_ARR_TIME'], bins = [0, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400], labels = ['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'])

dow_atime_df['dayOfWeek_binned'] = pd.cut(x = dow_atime_df['DAY_OF_WEEK'], bins = [0, 1, 2, 3, 4, 5 , 6, 7], labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

dow_atime_binned_df = dow_atime_df[['crs_arr_binned', 'dayOfWeek_binned', 'DEP_DEL30']].groupby(['crs_arr_binned', 'dayOfWeek_binned'], as_index = False).count()


(23906619, 3)

In [86]:
trace1 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_atime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Monday'].groupby('crs_arr_binned').sum()['DEP_DEL30'].tolist(),
    name='Monday'
    
)
trace2 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_atime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Tuesday'].groupby('crs_arr_binned').sum()['DEP_DEL30'].tolist(),
    name='Tuesday'
)

trace3 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_atime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Wednesday'].groupby('crs_arr_binned').sum()['DEP_DEL30'].tolist(),
    name='Wednesday'
    
)
trace4 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_atime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Thursday'].groupby('crs_arr_binned').sum()['DEP_DEL30'].tolist(),
    name='Thursday'
)

trace5 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_atime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Friday'].groupby('crs_arr_binned').sum()['DEP_DEL30'].tolist(),
    name='Friday'
    
)

trace6 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_atime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Saturday'].groupby('crs_arr_binned').sum()['DEP_DEL30'].tolist(),
    name='Saturday'
)

trace7 = go.Bar(
    x=['12am-2am', '2am-4am', '4am-6am', '6am-8am', '8am-10am', '10am-12pm', '12pm-2pm', '2pm-4pm', '4pm-6pm', '6pm-8pm', '8pm-10pm', '10pm-12am'],
    y= dow_atime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Sunday'].groupby('crs_arr_binned').sum()['DEP_DEL30'].tolist(),
    name='Sunday'
)

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7]
layout = go.Layout(
    barmode='group', 
    title="Day_Of_Week interacted with binned CRS_Arr_Time & Dep_Del30",
    xaxis=dict(title="Binned CRS Arrival Time", type='category'),
    yaxis=dict(title="Number of Delays")
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

### Day_Of_Week interacted with binned CRS_Elapsed_Time & Dep_Del30

In [88]:
dow_elapTime_df = full_data_dep[['DAY_OF_WEEK', 'CRS_ELAPSED_TIME', 'DEP_DEL30']].toPandas()

dow_elapTime_df['crs_elap_binned'] = pd.cut(x = dow_elapTime_df['CRS_ELAPSED_TIME'], bins = [0, 60, 120, 180, 240, 300, 360, 420, 480, 540, 600, 660, 720], labels = ['1 hour', '2 hours', '3 hours', '4 hours', '5 hours', '6 hours', '7 hours', '8 hours', '9 hours', '10 hours', '11 hours', '12 hours'])

dow_elapTime_df['dayOfWeek_binned'] = pd.cut(x = dow_elapTime_df['DAY_OF_WEEK'], bins = [0, 1, 2, 3, 4, 5 , 6, 7], labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

dow_elapTime_binned_df = dow_elapTime_df[['crs_elap_binned', 'dayOfWeek_binned', 'DEP_DEL30']].groupby(['crs_elap_binned', 'dayOfWeek_binned'], as_index = False).count()

In [89]:
trace1 = go.Bar(
    x=['1 hour', '2 hours', '3 hours', '4 hours', '5 hours', '6 hours', '7 hours', '8 hours', '9 hours', '10 hours', '11 hours', '12 hours'],
    y= dow_elapTime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Monday'].groupby('crs_elap_binned').sum()['DEP_DEL30'].tolist(),
    name='Monday'
    
)
trace2 = go.Bar(
    x=['1 hour', '2 hours', '3 hours', '4 hours', '5 hours', '6 hours', '7 hours', '8 hours', '9 hours', '10 hours', '11 hours', '12 hours'],
    y= dow_elapTime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Tuesday'].groupby('crs_elap_binned').sum()['DEP_DEL30'].tolist(),
    name='Tuesday'
)

trace3 = go.Bar(
    x=['1 hour', '2 hours', '3 hours', '4 hours', '5 hours', '6 hours', '7 hours', '8 hours', '9 hours', '10 hours', '11 hours', '12 hours'],
    y= dow_elapTime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Wednesday'].groupby('crs_elap_binned').sum()['DEP_DEL30'].tolist(),
    name='Wednesday'
    
)
trace4 = go.Bar(
    x=['1 hour', '2 hours', '3 hours', '4 hours', '5 hours', '6 hours', '7 hours', '8 hours', '9 hours', '10 hours', '11 hours', '12 hours'],
    y= dow_elapTime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Thursday'].groupby('crs_elap_binned').sum()['DEP_DEL30'].tolist(),
    name='Thursday'
)

trace5 = go.Bar(
    x=['1 hour', '2 hours', '3 hours', '4 hours', '5 hours', '6 hours', '7 hours', '8 hours', '9 hours', '10 hours', '11 hours', '12 hours'],
    y= dow_elapTime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Friday'].groupby('crs_elap_binned').sum()['DEP_DEL30'].tolist(),
    name='Friday'
    
)

trace6 = go.Bar(
    x=['1 hour', '2 hours', '3 hours', '4 hours', '5 hours', '6 hours', '7 hours', '8 hours', '9 hours', '10 hours', '11 hours', '12 hours'],
    y= dow_elapTime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Saturday'].groupby('crs_elap_binned').sum()['DEP_DEL30'].tolist(),
    name='Saturday'
)

trace7 = go.Bar(
    x=['1 hour', '2 hours', '3 hours', '4 hours', '5 hours', '6 hours', '7 hours', '8 hours', '9 hours', '10 hours', '11 hours', '12 hours'],
    y= dow_elapTime_binned_df[dow_dtime_binned_df['dayOfWeek_binned']=='Sunday'].groupby('crs_elap_binned').sum()['DEP_DEL30'].tolist(),
    name='Sunday'
)

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7]
layout = go.Layout(
    barmode='group', 
    title="Day_Of_Week interacted with binned CRS_Elapsed_Time & Dep_Del30",
    xaxis=dict(title="Binned CRS Elapsed Time", type='category'),
    yaxis=dict(title="Number of Delays")
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

### Holiday dataset

In [91]:
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import col
from pyspark.sql.functions import date_format
from pyspark.sql.functions import from_unixtime, unix_timestamp

departure_delay = (airlines
       .withColumn('FL_DATE', from_unixtime(unix_timestamp('FL_DATE', 'yyyy-MM-dd')))
       .withColumn("FL_DATE", date_format(col("FL_DATE"), "yyyy-MM-dd"))
       .filter('DEP_DELAY > 30')
       .groupBy("FL_DATE")
       .agg({"DEP_DELAY":"avg"})
      )

departure_delay_df = departure_delay.toPandas()

In [92]:
departure_delay_df.columns = ['FL_DATE', 'delay']
holiday_df = spark.read.csv("dbfs:/user/shajikk@ischool.berkeley.edu/scratch/" + 'holidays.csv').toPandas()
holiday_df.columns = ['ID', 'FL_DATE', 'holiday']
df = pd.merge(departure_delay_df, holiday_df, on='FL_DATE', how='outer', indicator=True)
df = df[df.delay.notnull()]
df['dt'] = df['FL_DATE'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))
df.sort_values(by='dt', ascending=True, inplace=True)
df.reset_index(drop=True,inplace=True)

Select Year 2019 for EDA

In [94]:
df = df[df.FL_DATE.str.contains('^2019')]

In [95]:
df_holidays = df.loc[df.holiday.notnull(),]
df_regular = df.loc[df.holiday.isnull(),]

fig = go.Figure()


fig.add_trace(go.Scatter(
  mode = 'markers',
  x = df_holidays.dt.to_list(),
  y = df_holidays.delay.to_list(),
  hovertext=[i[0] + ', ' + str(round(i[1], 2)) + '<br>'+ i[2] for i in list(zip(df_holidays.FL_DATE.to_list(), df_holidays.delay.to_list(), df_holidays.holiday.to_list()))],
  hoverinfo="text",
  marker = dict(
   color = 'rgb(17, 157, 255)',
   size = 12,
   line = dict(
     color = 'rgb(231, 99, 250)',
     width = 2
   )
  ),
  showlegend = False,
  name = '', 
  
))

fig.add_trace(go.Scatter(
  mode = 'markers',
  x = df_regular.dt.to_list(),
  y = df_regular.delay.to_list(),
  hovertext = [i+ ', ' + str(round(j, 2)) for i, j in list(zip(df_regular.FL_DATE.to_list(), df_regular.delay.to_list()))],
  hoverinfo="text",
  marker = dict(
   color = 'rgb(17, 157, 255)',
   size = 3,
  ),
  showlegend = False,
  name = ''
))

fig.add_trace(go.Scatter(
  mode = 'lines',
  line_shape='spline',
  x = df.dt.to_list(),
  y = df.delay.to_list(),
  hovertext=df_holidays.holiday.to_list(),
  showlegend = False,
  name = '', 
  
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
#fig.update_traces(mode='lines+markers')
fig.update_layout( 
                  xaxis=dict(categoryorder='total descending', title='Day of the year'), 
                  title=go.layout.Title(text="<b>Airline departure delay overlyed on holidays (for the year 2019)</b>"),  
                  yaxis=dict(title='Airline departure delays > 30 mins'))
fig.update_layout(width=1000, height=600)
fig.show()